In [1]:
import os
import sys
import warnings

warnings.filterwarnings("ignore")

sys.path.append(f"{os.getcwd()}/../")

In [2]:
from string import punctuation
import pandas as pd
from nltk import word_tokenize


class EmotionNaiveClassifier:
    
    def __init__(self, path=""):
        self.alegria = pd.read_csv(f"{path}alegria", header=None)
        self.surpresa = pd.read_csv(f"{path}surpresa", header=None)
        self.confianca = pd.read_csv(f"{path}confianca", header=None)
        self.desgosto = pd.read_csv(f"{path}desgosto", header=None)
        self.medo = pd.read_csv(f"{path}medo", header=None)
        self.raiva = pd.read_csv(f"{path}raiva", header=None)
        self.tristeza = pd.read_csv(f"{path}tristeza", header=None)
        
        self.alegria.columns = ["emocao"]
        self.surpresa.columns = ["emocao"]
        self.confianca.columns = ["emocao"]
        self.desgosto.columns = ["emocao"]
        self.medo.columns = ["emocao"]
        self.raiva.columns = ["emocao"]
        self.tristeza.columns = ["emocao"]
    
        self.dataset = list(zip(
            ["alegria", "surpresa", "confiança", "desgosto", "medo", "raiva", "tristeza"],
            [self.alegria, self.surpresa, self.confianca, self.desgosto, self.medo, self.raiva, self.tristeza]
        ))
    
    def predict(self, phrase):
        coef_ = {
            "alegria": 0,
            "surpresa": 0,
            "confiança": 0,
            "desgosto": 0,
            "medo": 0,
            "raiva": 0,
            "tristeza": 0
        }
        verificar_negacao = True if "não" in phrase or "sem" in phrase else False
        
        phrase = [word for word in word_tokenize(phrase) if word not in punctuation]
        for i, word in enumerate(phrase):
            for sent, df in self.dataset:
                wd_ = df[df["emocao"] == word]
                if wd_.size > 0:
                    negando = False
                    
                    if verificar_negacao:
                        for m in range(5):
                            if i-m >= 0:
                                if phrase[i-m] == "não" or phrase[i-m] == "sem":
                                    negando = True
                                    break

                    if not negando:
                        coef_[sent] += 1
                    else:
                        if sent == "surpresa":
                            coef_["tristeza"] += 0.25
                        elif sent == "confiança":
                            coef_["tristeza"] += 0.15
                            coef_["medo"] += 0.25
                        elif sent == "alegria":
                            coef_["tristeza"] += 0.15
                            coef_["desgosto"] += 0.25
                        elif sent == "desgosto":
                            coef_["alegria"] += 0.25
                        elif sent == "tristeza":
                            coef_["alegria"] += 0.25
                            coef_["confiança"] += 0.15

        coef_ = {sent: round(val, 3) for sent, val in coef_.items()}
        coef_["tristeza"] += coef_["medo"] * 1e-1
        coef_["confiança"] -= coef_["medo"] * 1e-1
        coef_["confiança"] = 0.0 if coef_["confiança"] < 0.0 else coef_["confiança"]
        coef_["raiva"] += coef_["medo"] * 1e-2
        coef_["raiva"] += coef_["desgosto"] * 1e-2
        coef_["alegria"] += coef_["confiança"] * 1e-2
        coef_["alegria"] += coef_["surpresa"] * 1e-2
        coef_ = {sent: round((val / (100 / len(phrase))) * 100, 2) for sent, val in coef_.items()}
        return coef_

In [3]:
frases = [
    "minha mãe esta doente e não sei mais o que fazer, preciso de ajuda",
    "acho que tudo dará certo, tenho certeza que esse é o caminho correto.",
    "não quero estragar a surpresa, mas você esta muito equivocado.",
    "espero que você morra seu desgraçado",
    "eu não gosto de você, seu bastardo.",
    "o lobo mau, vai comer a chapéuzinho vermelho e somento o caçador poderá salva-la",
    "eu gostei muito de te conhecer, você é uma pessoa bem legal",
    "esta tudo bem, sem problemas",
    "sem você eu não sou nada",
    "nossa que festa legal hein",
    "trabalho excelente, meus parabéns",
    "este trabalho foi realizado com muita atenção, parabéns, ficou excelente"
]

clf = EmotionNaiveClassifier(path=f"{os.getcwd()}/../data/emocoes/")
for frase in frases:
    print(frase)
    print(clf.predict(frase))
    print()

minha mãe esta doente e não sei mais o que fazer, preciso de ajuda
{'alegria': 0.11, 'surpresa': 0.0, 'confiança': 11.2, 'desgosto': 14.0, 'medo': 28.0, 'raiva': 0.42, 'tristeza': 16.8}

acho que tudo dará certo, tenho certeza que esse é o caminho correto.
{'alegria': 0.38, 'surpresa': 0.0, 'confiança': 37.7, 'desgosto': 0.0, 'medo': 13.0, 'raiva': 0.13, 'tristeza': 1.3}

não quero estragar a surpresa, mas você esta muito equivocado.
{'alegria': 0.0, 'surpresa': 0.0, 'confiança': 0.0, 'desgosto': 10.0, 'medo': 0.0, 'raiva': 0.1, 'tristeza': 2.5}

espero que você morra seu desgraçado
{'alegria': 0.0, 'surpresa': 0.0, 'confiança': 0.0, 'desgosto': 6.0, 'medo': 0.0, 'raiva': 12.06, 'tristeza': 0.0}

eu não gosto de você, seu bastardo.
{'alegria': 0.0, 'surpresa': 0.0, 'confiança': 0.0, 'desgosto': 8.75, 'medo': 0.0, 'raiva': 7.09, 'tristeza': 1.05}

o lobo mau, vai comer a chapéuzinho vermelho e somento o caçador poderá salva-la
{'alegria': 0.0, 'surpresa': 0.0, 'confiança': 0.0, 'desgost